# Sandbox

## Config

In [158]:
# Import dependencies
import tekore as tk
import spotipy
import os
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
# Set global variables
MINDATE = datetime.today() - relativedelta(years=3)


## Authentication

In [8]:
### Tekore
# Get client token
app_token = tk.request_client_token(CLIENTID, CLIENTSECRET)
# Create spotify instance
spotify = tk.Spotify(app_token)

## Tekore Sandbox

In [78]:
# First let's try to get an artist's Spotify ID by searching for their name
# Searching by artist name returns a tuple with many different Spotify elements
artists = spotify.search('Sylvan Esso', types=['artist'], limit=10)
artists

FullArtistOffsetPaging with fields:
  href = 'https://api.spotify.com/v1/search?query=Sylvan+Esso&type=arti...'
  items = [10 x FullArtist(external_urls, followers, genres, href, ...)]
  limit = 10
  next = 'https://api.spotify.com/v1/search?query=Sylvan+Esso&type=arti...'
  offset = 0
  previous = None
  total = 60

In [61]:
# The code above doesn't help get the artist's id because it returns a tuple and we cannot select the tuple's items like we would a list
artists.items[0]

AttributeError: 'tuple' object has no attribute 'items'

In [79]:
# The code below, though, does allow us to unpack the tuple
# Note the comma trailing the variable
# This tells python we know we are dealing with a tuple, and we can select the first artist from it
artists, = spotify.search("Sylvan Esso", types=['artist'], limit=10)
artists.items[0]

FullArtist with fields:
  external_urls = {'spotify'}
  followers = Followers(href, total)
  genres = [4 x str]
  href = 'https://api.spotify.com/v1/artists/39vA9YljbnOApXKniLWBZv'
  id = '39vA9YljbnOApXKniLWBZv'
  images = [3 x Image(height, url, width)]
  name = 'Sylvan Esso'
  popularity = 60
  type = 'artist'
  uri = 'spotify:artist:39vA9YljbnOApXKniLWBZv'

In [80]:
# From within the artist item we can select other elements like genre or artist_id
print("Sylvan Esso is classified under the following genres: " + ", ".join([x for x in artist.genres]))
print(f"Sylvan Esso's Spotify ID is: {artist.id}")

Sylvan Esso is classified under the following genres: electropop, etherpop, indie pop, indie rock
Sylvan Esso's Spotify ID is: 39vA9YljbnOApXKniLWBZv


In [83]:
# Let's see what the artist API can tell us
# We can see the artist method again returns the FullArtist
sylvanEsso = spotify.artist('39vA9YljbnOApXKniLWBZv')
sylvanEsso
# Interesting that we can get popularity on a scale of 0-100; this will be useful later

FullArtist with fields:
  external_urls = {'spotify'}
  followers = Followers(href, total)
  genres = [4 x str]
  href = 'https://api.spotify.com/v1/artists/39vA9YljbnOApXKniLWBZv'
  id = '39vA9YljbnOApXKniLWBZv'
  images = [3 x Image(height, url, width)]
  name = 'Sylvan Esso'
  popularity = 60
  type = 'artist'
  uri = 'spotify:artist:39vA9YljbnOApXKniLWBZv'

In [104]:
# Let's see if we can get a list of ids for the artist's albums
sylvanAlbums = spotify.artist_albums('39vA9YljbnOApXKniLWBZv', limit=50)
print(len(sylvanAlbums.items))

50


In [124]:
# We have a problem: Spotify limits the number of albums to 50. 
# We will have to write some logic to get all the albums if there are more than 50
# The offset parameter will help
sylvanAlbums = spotify.artist_albums('39vA9YljbnOApXKniLWBZv', limit=50, offset=50)
print(len(sylvanAlbums.items))
# We can see there are 87 albums (50+37) as of this writing

37


In [125]:
# We can use a list iteration to get all the album ids
[x.id for x in sylvanAlbums.items][:5]

['3EOSrgN0wOOqNvS5GUsiLp',
 '6w0oK4Y6MvRqFaE3J0Uwyz',
 '0oTOOEPtYGZFYnWlLnOsxL',
 '3BBCxnIvQauRM4FT2Mc3pX',
 '1Aus1dFkP2BSyzibCl0HA7']

In [163]:
# Let's make a function to join the multiple pages of albums together
def get_artistAlbums(artistID):
    # Initialize variables
    length = 50
    albumList = []
    offset=0
    # If the returned page is 50 items that means we need to call the API for another chunk
    while length==50:
        # Call the API to get the artist's albums
        albums = spotify.artist_albums(artistID, limit=50, offset=offset)
        # Append this chunk to our existing list
        albumList = albumList + [x.id for x in albums.items]
        # Reset the length. If the length goes below 50 we will not call the API again
        length = len(albums.items)
        # Increase the offset so we are not getting the same albums again
        offset+=50
    return albumList

# It's working! Now we have all 87 albumIDs
sylvanAlbumList = get_artistAlbums('39vA9YljbnOApXKniLWBZv')
print(len(sylvanAlbumList))
sylvanAlbumList[:5]


87


['6rl4rzZxECegl9PdYG0hfk',
 '2KAPJ40On8JY1Yt64JJgTk',
 '68ClkAMsCDyx2lQMsnbY55',
 '1BJMCEXQ7JmuVlJ6cYbe3x',
 '1862qnxqsLeiNDblknjtiL']

In [159]:
# One of the requirements of a summer playlist is that all songs must be newer than three years old 
# We can include this logic in our list comprehension
[x.id for x in sylvanAlbums.items if datetime.strptime(x.release_date, '%Y-%m-%d')>MINDATE]

['3EOSrgN0wOOqNvS5GUsiLp',
 '6w0oK4Y6MvRqFaE3J0Uwyz',
 '0oTOOEPtYGZFYnWlLnOsxL',
 '3BBCxnIvQauRM4FT2Mc3pX',
 '1Aus1dFkP2BSyzibCl0HA7',
 '0duHxSsZOXo0rlxY2ZabO0',
 '3IBo4lqI2xcSEkEMl73Aop',
 '0LX2FkR9ARLv5PmVQ211jZ',
 '61PjEcLjGh1UvfsCkn5eB8',
 '5qrhRBImmdLDEm2W8Y4lE3',
 '3hJ8Qe8dTlKJwAt1VPNc7g',
 '6tfc4x6M0rRoizzWXggXGa',
 '5N5q1BjX8YE0hjhBnwJRMq',
 '00wKAsbWtTgG5uCq524Tvk',
 '6fM5eqkTrd7ReiwRPnRMzg',
 '1UKlb2Lb7CW1trU8ultvGr']

In [190]:
# We can also specify in the API call that we only want actual albums or singles
# Let's try this again. Make a function to join the multiple pages of albums together
def get_artistAlbums(artistID):
    # Initialize variables
    length = 50
    albumList = []
    offset=0
    # If the returned page is 50 items that means we need to call the API for another chunk
    while length==50:
        # Call the API to get the artist's albums
        albums = spotify.artist_albums(artistID, include_groups=['album'], limit=50, offset=offset)
        # Append this chunk to our existing list
        albumList = albumList + [x.id for x in albums.items if datetime.strptime(x.release_date, '%Y-%m-%d')>MINDATE]
        # Reset the length. If the length goes below 50 we will not call the API again
        length = len(albums.items)
        # Increase the offset so we are not getting the same albums again
        offset+=50
    return albumList

# Now we only have 15 albumIDs that are in the dates that we want and the type that we want
sylvanAlbumList = get_artistAlbums('39vA9YljbnOApXKniLWBZv')
print(len(sylvanAlbumList))
sylvanAlbumList[:5]


3


['6rl4rzZxECegl9PdYG0hfk', '2KAPJ40On8JY1Yt64JJgTk', '68ClkAMsCDyx2lQMsnbY55']

In [191]:
# Next let's get the top 5 tracks from these albums
pops = [spotify.album(x).popularity for x in sylvanAlbumList]

In [192]:
names = [spotify.album(x).name for x in sylvanAlbumList]


In [193]:
total = zip(names, pops)
list(total)

[('No Rules Sandy', 49), ('Free Love', 49), ('WITH', 32)]

In [201]:
[x.id for x in spotify.artist_top_tracks('39vA9YljbnOApXKniLWBZv', 'US')[:5]]

['30GGIrrJdSNtecPiFcVP5O',
 '1BRwuvjhkgezmv1gcI6lT6',
 '5Y42u37PlyQXegnuI1Hpo1',
 '7bp7QInQ5uIMzYjJqTtjVA',
 '0idjTHGhGRBdWTmsalq3tF']